In [2]:
import os


In [3]:
%pwd

'c:\\Users\\udita\\OneDrive - The University of Texas at Dallas\\Personal\\UT Dallas\\Project\\Machine-Learning-Project-with-MLFlow\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'c:\\Users\\udita\\OneDrive - The University of Texas at Dallas\\Personal\\UT Dallas\\Project'

In [7]:
os.chdir("c:\\Users\\udita\\OneDrive - The University of Texas at Dallas\\Personal\\UT Dallas\\Project\\Machine-Learning-Project-with-MLFlow")

In [8]:
%pwd

'c:\\Users\\udita\\OneDrive - The University of Texas at Dallas\\Personal\\UT Dallas\\Project\\Machine-Learning-Project-with-MLFlow'

In [13]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [10]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories
from mlProject.entity.config_entity import DataIngestionConfig

In [14]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [16]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [29]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
            if not os.path.exists(self.config.local_data_file):
                filename, headers = request.urlretrieve(
                    url = self.config.source_URL,
                    filename = self.config.local_data_file
                )
                logger.info(f"{filename} download! with following info: \n{headers}")
            else:
                logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [30]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-09-04 10:57:23,333: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-09-04 10:57:23,341: INFO: common: yaml file: params.yaml loaded successfully]
[2023-09-04 10:57:23,357: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-09-04 10:57:23,362: INFO: common: created directory at: artifacts]
[2023-09-04 10:57:23,363: INFO: common: created directory at: artifacts/data_ingestion]
[2023-09-04 10:57:24,168: INFO: 2383606181: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 6338:366B:7E6EF:91CE7:64F5FE63
Accept-Ranges: bytes
Date: Mon, 04 Sep